## Web Data Analytics - Web Scraping

In [1]:
from platform import python_version

In [2]:
print(python_version())

3.9.13


In [3]:
#imports
import re
import bs4
import time
import json
import requests
from bs4 import BeautifulSoup as bs

In [4]:
%reload_ext watermark
%watermark --iversions

requests: 2.28.1
json    : 2.0.9
re      : 2.2.1
bs4     : 4.11.1



##### Funções Para Extração de Dados Via Web Scraping

In [5]:
# Função para obter tudo que estiver nas tags td e div e com seletor CSS spaceit_pad
# Assim extraímos informações gerais da página
def getInfoGeral(soup):
    information = soup.select("td div .spaceit_pad")
    side_info_par = []
    for info in information:
        side_info_par.append(info.text.split())
    return side_info_par

In [6]:
# Função para obter o número de pessoas que adicionaram o anime aos favoritos
def getFavorite(information):
    for info in information:
        if info[0] == "Favorites:":
            data = info[1].split(",")
            return int("".join(data))
    return None

In [7]:
# Função para obter o número do episódio
def episodeNumberParser(information):
    for info in information:
        if (info[0] == "Episodes:"):
            if info[1] != "Unknown":
                return int(info[1])
            return None
    return None

In [8]:
# Função para obter o tipo de anime TV
def typeParser(information):
    for info in information:
        if (info[0] == "Type:"):
            return info[1]
    return None

In [9]:
# Função para obter o dia em que o anime estreou
def dayPremiered(information):
    for info in information:
        if (info[0] == "Premiered:"):
            premired_day = ""
            return " ".join(info[1:])
    return None

In [10]:
# Função para obter a fonte de onde do anime
def getSource(information):
    for info in information:
        if info[0] == "Source:":
            return " ".join(info[1:])
    return None

In [11]:
# Função para retornar uma lista de gêneros do anime
def getGenres(information):
    for info in information:
        if info[0] == "Genres:":
            genres = "".join(info[1:])
            return genres.split(",")
    return None

In [12]:
# Função que retorna uma lista de estúdios que estão patrocinando este anime
def getStudios(information):
    for info in information:
        if info[0] == "Studios:":
            return " ".join(info[1:])
    return None

In [13]:
# Função que retorna uma lista de empresas que realmente possuem a licença deste anime
def getLicense(information):
    for info in information:
        if info[0] == "Licensors:":
            if info[1] == "None":
                return None
            license = "".join(info[1:])
            return license.split(",")
    return None

In [14]:
# Função que obtém a classificação do filme 
def getRating(information):
    for info in information:
        if info[0] == "Rating:":
            return "".join(info[1:4])
    return None

In [15]:
# Função para obter o nome de empresas que estão financiando o anime (Producers)
def producers_names(soup):
    div_and_a = soup.find_all(["div","a"], class_=["spaceit_pad"])
    for index, j in enumerate(div_and_a):
        if div_and_a[index].text.split("\n")[1] == "Producers:":
            res = re.sub(" *", "", div_and_a[index].text)
            new_res = res.split("\n")
            for i,j in enumerate(new_res):
                if j == "Producers:":
                    return new_res[i+1].split(",")
                    break
    return None

In [16]:
# Função para obter informações de ranking
def getRanking(soup):
    ranking = soup.select_one(".ranked strong").text
    data = ranking.split("#")[1]
    return int(data)

In [17]:
# Função para obter o score do Anime
def getScore(soup):
    data = float(soup.select_one(".score-label").text)
    return data

In [18]:
# Função para obter informação de popularidade
def getPopularity(soup):
    popularity = soup.select_one(".popularity strong").text
    data = popularity.split("#")[1]
    return int(data)

In [19]:
# Função para obter informação sobre os membros
def getMembers(soup):
    members = soup.select_one(".members strong").text
    data = members.split(",")
    return int("".join(data))

In [20]:
# Função para obter o número de pessoas que votaram no anime
def votersCount(soup):
    count = soup.select(".js-statistics-info span")
    data = []
    for i in count:
        data.append(i.text)
    if len(data) > 3:
        return int(data[2])
    return None

In [21]:
# Função para obter a voz do ator que faz a narração
def voiceActorsName(soup):
    actorNames = []
    names = soup.select(".js-anime-character-va-lang .spaceit_pad a")
    for name in names:
        data = "".join(name.text.split())
        actorNames.append(data)
    return actorNames

In [22]:
# Função para obter o idioma do ator que faz a narração
def voiceActorsLanguage(soup):
    languages = []
    langNames = soup.select(".js-anime-character-language")
    for language in langNames:
        lang = "".join(language.text.split())
        languages.append(lang)
    return languages

In [23]:
# Função que usa o voiceActorName e voiceActorsLanguage para criar um dataframe 
# que irá anexar todas as informações relacionadas às vozes dos atores.
# Usando essas informações, dividiremos os voiceActors com base em seus idiomas.
def voiceActorNameandLanguage(soup):
    names = voiceActorsName(soup)
    languages = voiceActorsLanguage(soup)
    namesAndLang = []
    for i in range(len(names)):
        namesAndLang.append([names[i], languages[i]])
    return namesAndLang

In [24]:
# Função que retorna a ocupação de cada funcionário
def staffOccupation(soup):
    data = []
    for i in soup.select('td .js-scrollfix-bottom-rel .spaceit_pad small'):
        data.append("".join(i.text.split()))
    return data

In [25]:
# Função que retorna o nome dos funcionários
def staffName(soup):
    data = []
    for i in soup.select('td .js-scrollfix-bottom-rel a'):
        data.append("".join(i.text.split()))
    uncleanData = []
    for index, info in enumerate(data):
        if info == "Addstaff":
            uncleanData.append(data[index+1:])
            break
    cleanData = []
    for i in uncleanData:
        for j in i:
            if len(j) > 1:
                cleanData.append(j)
    return cleanData

In [26]:
# Função que concatena nome e ocupação dos funcionários
def staffNameandOccupation(soup):
    name = staffName(soup)
    occupation = staffOccupation(soup)
    nameAndOccupation = []
    for i in range(len(name)):
        nameAndOccupation.append([name[i], occupation[i]])
    return nameAndOccupation

In [27]:
# Função que retorna o nome do Anime
def animeName(soup):
    name = soup.select_one(".title-name").text
    if len(name) > 1:
        return name
    return None

##### Web Scraping : https://myanimelist.net/topanime.php?type=airing&limit=100

In [28]:
%%time

for limit in range(0, 451, 50):
    try:
        links = []

        url = f'https://myanimelist.net/topanime.php?limit={limit}'

        rq = requests.get(url)
        print(rq.status_code)

        soup = bs(rq.text, 'html.parser')

        listsofA = soup.select('.hoverinfo_trigger a')

        for a in listsofA:
            links.append(a.get('href'))

        file = open(f'projeto7_analise_de_dados_pyton/dados/animeLinks{limit}.txt', 'w')

        for link in links:
            file.writelines(link+'\n')

        file.close()

        time.sleep(10)
    except:
        continue

200
200
200
200
200
200
200
200
200
200
Wall time: 47.8 s


In [29]:
%%time
for limit in range(0, 51, 50):
    
    # Lista
    animeInfo = []
    
    # Leitura dos arquivos txt com os links
    with open(f"projeto7_analise_de_dados_pyton/dados/animeLinks{limit}.txt", "r") as file:
        links = file.readlines()
    
    print("Fazendo Web Scraping. Seja paciente e aguarde...")
    
    # Request para cada link
    for link in links:
        
        # Exemplo de url: https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
        rq = requests.get(link)
        
        # Exemplo de url: https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/characters
        rq2 = requests.get(link[:-2] + "/characters")
        
        # Soup com parse do html
        soup = bs(rq.text, "html.parser")
        soup2 = bs(rq2.text, "html.parser")
        
        # Obtém info geral da página
        information = getInfoGeral(soup)
        
        # Extrai os dados e salva na lista no formato de dicionário
        animeInfo.append(dict(
            name                      = animeName(soup),
            producers                 = producers_names(soup),
            ranking                   = getRanking(soup),
            votersCount               = votersCount(soup),
            score                     = getScore(soup),
            popularity                = getPopularity(soup),
            members                   = getMembers(soup),
            favorite                  = getFavorite(information),
            episodes                  = episodeNumberParser(information),
            animeType                 = typeParser(information),
            premiredDate              = dayPremiered(information),
            animeSource               = getSource(information),
            genres                    = getGenres(information),
            studios                   = getStudios(information),
            licenses                  = getLicense(information),
            rating                    = getRating(information),
            voiceActorNameandLanguage = voiceActorNameandLanguage(soup2),
            staffNameandOccupation    = staffNameandOccupation(soup2)))
        
        # Sleep
        time.sleep(10)

    # Grava o resultado em disco
    with open(f"projeto7_analise_de_dados_pyton/dados/animeInfo{limit}.txt", 'w') as fout:
        json.dump(animeInfo, fout)
        
    # Sleep
    time.sleep(10)

Fazendo Web Scraping. Seja paciente e aguarde...
Fazendo Web Scraping. Seja paciente e aguarde...
Wall time: 21min 18s


In [30]:
%%time
for limit in range(0, 51, 50):
    
    # Lista
    animeInfoName = []
    
    # Abre os arquivos de links para leitura
    with open(f"projeto7_analise_de_dados_pyton/dados/animeLinks{limit}.txt", "r") as file:
        links = file.readlines()
    
    print("Extraindo os nomes dos animes. Seja paciente e aguarde...")
    
    # Loop pelos links para extrair o nome dos animes
    for link in links:
        rq = requests.get(link)
        soup = bs(rq.text, "html.parser")
        animeInfoName.append(dict(name = animeName(soup)))
        time.sleep(5)

    # Salva o resultado em disco no formato json
    with open(f"projeto7_analise_de_dados_pyton/dados/animeInfoNames{limit}.txt", 'w') as fout:
        json.dump(animeInfoName, fout)

Extraindo os nomes dos animes. Seja paciente e aguarde...
Extraindo os nomes dos animes. Seja paciente e aguarde...
Wall time: 10min 31s
